In [1]:
import os    
#os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu0,floatX=float32"
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cuda,floatX=float32"

import numpy as np
from sklearn.utils import shuffle
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from PIL import Image
import argparse
import math

from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Input
from keras.layers import Dropout
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D, UpSampling3D
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Deconvolution2D
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau

from keras.layers.core import Flatten
from keras.constraints import maxnorm

import keras.regularizers

from keras.optimizers import SGD, Adam
from keras.datasets import mnist
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator




Using Theano backend.


Couldn't import dot_parser, loading of dot files will not be possible.


/home/mrafi123/anaconda3/lib/python3.5/site-packages/theano/gpuarray/dnn.py:135: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to version 5.1.
  warnings.warn("Your cuDNN version is more recent than "
Using cuDNN version 6021 on context None
Mapped name None to device cuda: GeForce 920M (0000:08:00.0)


In [2]:
img_rows = 28
img_cols = 28


In [3]:
#test for gpu
from theano import function, config, shared, tensor
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], tensor.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, tensor.Elemwise) and
              ('Gpu' not in type(x.op).__name__)
              for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

[GpuElemwise{exp,no_inplace}(<GpuArrayType<None>(float32, (False,))>), HostFromGpu(gpuarray)(GpuElemwise{exp,no_inplace}.0)]
Looping 1000 times took 0.874952 seconds
Result is [ 1.23178029  1.61879349  1.52278066 ...,  2.20771813  2.29967761
  1.62323296]
Used the gpu


In [86]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')



x_train shape: (60000, 28, 28)
60000 train samples
10000 test samples


In [87]:
from sklearn.model_selection import train_test_split

l_images_train, u_images_train, l_labels_train, u_labels_train = train_test_split(
    x_train,y_train,test_size=59000, random_state=42)

In [88]:
print (l_images_train.shape)
print (u_images_train.shape)
print (l_labels_train.shape)
print (u_labels_train.shape)


(1000, 28, 28)
(59000, 28, 28)
(1000,)
(59000,)


In [89]:
unique, counts = numpy.unique(l_labels_train, return_counts=True)
print (dict(zip(unique, counts)))

{0: 105, 1: 114, 2: 107, 3: 92, 4: 90, 5: 83, 6: 103, 7: 100, 8: 102, 9: 104}


In [90]:
print(l_images_train.shape)


(1000, 28, 28)


In [91]:
print (l_images_train[0])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  22 151
  223 238  56   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  74 214 236
  134  96  19   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0 104 248 212  49
    0  43   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0  57 245 21

In [92]:

l_images_train = l_images_train.astype('float32')
l_images_train /= 255
print(l_images_train.shape)

print (l_images_train.shape)
l_images_train = l_images_train.reshape(-1,1, 28, 28)



(1000, 28, 28)
(1000, 28, 28)


In [93]:
print (l_images_train.shape)


(1000, 1, 28, 28)


In [94]:
print (l_labels_train.shape)
l_labels_train = l_labels_train.reshape(1000,1)
print (l_labels_train.shape)


(1000,)
(1000, 1)


In [95]:
print (l_labels_train)

[[9]
 [4]
 [2]
 [6]
 [0]
 [5]
 [0]
 [9]
 [7]
 [0]
 [3]
 [0]
 [8]
 [6]
 [8]
 [0]
 [0]
 [2]
 [9]
 [1]
 [9]
 [7]
 [0]
 [9]
 [8]
 [9]
 [6]
 [7]
 [4]
 [6]
 [3]
 [1]
 [4]
 [8]
 [8]
 [9]
 [8]
 [7]
 [9]
 [6]
 [6]
 [2]
 [1]
 [2]
 [1]
 [8]
 [0]
 [2]
 [9]
 [8]
 [7]
 [9]
 [4]
 [2]
 [0]
 [0]
 [6]
 [6]
 [0]
 [0]
 [5]
 [9]
 [4]
 [1]
 [0]
 [0]
 [8]
 [9]
 [1]
 [9]
 [3]
 [8]
 [4]
 [7]
 [3]
 [4]
 [8]
 [4]
 [8]
 [0]
 [7]
 [7]
 [8]
 [3]
 [7]
 [9]
 [2]
 [1]
 [6]
 [7]
 [5]
 [0]
 [2]
 [5]
 [7]
 [3]
 [4]
 [4]
 [5]
 [0]
 [6]
 [1]
 [3]
 [4]
 [0]
 [6]
 [9]
 [5]
 [5]
 [6]
 [2]
 [6]
 [0]
 [8]
 [9]
 [7]
 [9]
 [1]
 [7]
 [4]
 [2]
 [0]
 [7]
 [5]
 [3]
 [4]
 [7]
 [0]
 [2]
 [1]
 [8]
 [0]
 [2]
 [1]
 [4]
 [7]
 [2]
 [2]
 [1]
 [3]
 [5]
 [2]
 [1]
 [0]
 [5]
 [2]
 [5]
 [4]
 [4]
 [4]
 [2]
 [2]
 [9]
 [4]
 [9]
 [2]
 [3]
 [2]
 [7]
 [0]
 [2]
 [5]
 [0]
 [4]
 [1]
 [7]
 [8]
 [5]
 [2]
 [2]
 [7]
 [4]
 [8]
 [1]
 [9]
 [6]
 [5]
 [7]
 [2]
 [6]
 [8]
 [5]
 [4]
 [0]
 [9]
 [0]
 [6]
 [4]
 [5]
 [1]
 [0]
 [0]
 [6]
 [1]
 [5]
 [8]
 [1]
 [8]
 [3]
 [1]


In [96]:
l_labels_train = np_utils.to_categorical(l_labels_train, 10)

In [97]:
print (l_labels_train.shape)


(1000, 10)


In [98]:
# define vars

input_shape = (784,)
input_reshape = (1,28, 28)

conv_num_filters = 5
conv_filter_size = 5

pool_size = (2, 2)

hidden_num_units = 50
output_num_units = 10

epochs = 5
batch_size = 128

In [99]:
feature_layers = [
    Convolution2D(25, 5, 5, border_mode='same', input_shape=(input_reshape), activation='relu'),
    MaxPooling2D(pool_size=pool_size),
    Convolution2D(25, 5, 5, border_mode='same',  activation='relu'),
    MaxPooling2D(pool_size=pool_size),
    Convolution2D(25, 4, 4, border_mode='same', activation='relu'),
    Flatten(),
    Dense(output_dim=hidden_num_units, activation='relu')
]

classification_layers = [
    Dense(10, activation='softmax', W_regularizer=keras.regularizers.l2(0.01), name='fc_layer2')
]



In [100]:

model = Sequential(feature_layers + classification_layers)


In [101]:
print (model.summary())


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_7 (Convolution2D)  (None, 25, 28, 28)    650         convolution2d_input_3[0][0]      
____________________________________________________________________________________________________
maxpooling2d_5 (MaxPooling2D)    (None, 25, 14, 14)    0           convolution2d_7[0][0]            
____________________________________________________________________________________________________
convolution2d_8 (Convolution2D)  (None, 25, 14, 14)    15650       maxpooling2d_5[0][0]             
____________________________________________________________________________________________________
maxpooling2d_6 (MaxPooling2D)    (None, 25, 7, 7)      0           convolution2d_8[0][0]            
___________________________________________________________________________________________

In [102]:
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

print('Model Compilation successful')

Model Compilation successful


In [103]:
model.fit(l_images_train, l_labels_train, batch_size=batch_size, nb_epoch=10,
          verbose=1)


Epoch 1/10
1000/1000 [==============================] - 0s - loss: 2.3923 - acc: 0.2720     
Epoch 2/10
1000/1000 [==============================] - 0s - loss: 1.8704 - acc: 0.5410     
Epoch 3/10
1000/1000 [==============================] - 0s - loss: 1.0900 - acc: 0.7400     
Epoch 4/10
1000/1000 [==============================] - 0s - loss: 0.7362 - acc: 0.8180     
Epoch 5/10
1000/1000 [==============================] - 0s - loss: 0.5942 - acc: 0.8560     
Epoch 6/10
1000/1000 [==============================] - 0s - loss: 0.5101 - acc: 0.8860     
Epoch 7/10
1000/1000 [==============================] - 0s - loss: 0.4312 - acc: 0.9170     
Epoch 8/10
1000/1000 [==============================] - 0s - loss: 0.3736 - acc: 0.9300     
Epoch 9/10
1000/1000 [==============================] - 0s - loss: 0.3309 - acc: 0.9490     
Epoch 10/10
1000/1000 [==============================] - 0s - loss: 0.2986 - acc: 0.9600     


In [104]:
# Prepare test data
x_test = x_test.astype('float32')
x_test /= 255
print(x_test.shape)

print (x_test.shape)
x_test = x_test.reshape(-1,1, 28, 28)
print (x_test.shape)


(10000, 28, 28)
(10000, 28, 28)
(10000, 1, 28, 28)


In [105]:
y_test

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

In [106]:
y_test = np_utils.to_categorical(y_test, 10)

In [107]:
y_test

array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [108]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.395036410713
Test accuracy: 0.9233


In [109]:
x_test[1]

array([[[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.  

In [110]:
y_pred = model.predict(x_test)

In [111]:
y_pred[0]

array([  3.00788179e-05,   4.78546781e-07,   7.58414387e-07,
         1.33511621e-05,   9.07759379e-09,   4.69311459e-07,
         8.42060810e-10,   9.99808729e-01,   3.32009472e-06,
         1.42846038e-04], dtype=float32)

In [112]:
y_test[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.])

In [113]:
y_pred[200]

array([  3.54258627e-05,   1.23669146e-08,   6.35241298e-03,
         9.92520213e-01,   1.63691860e-09,   3.26768495e-05,
         1.49934393e-04,   3.34126782e-10,   9.08836257e-04,
         5.24322274e-07], dtype=float32)

In [114]:
y_test[4]

array([ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.])

In [115]:
np.argmax(y_pred[4])

4

In [116]:
y_pred_new = y_pred[:, :10]  # First 10 columns of full_array


In [117]:
y_pred_new[1]

array([  6.07653146e-06,   6.06766029e-04,   9.63414073e-01,
         2.74561439e-03,   7.47198392e-09,   1.99802544e-05,
         3.31398025e-02,   4.94708718e-10,   6.76963245e-05,
         1.07124558e-08], dtype=float32)

In [118]:
y_pred_new = np.ndarray.argmax(y_pred_new,axis=1)

In [119]:
y_pred_new.shape

(10000,)

In [120]:
y_pred_new

array([7, 2, 1, ..., 4, 5, 6])

In [121]:
y_test_new = np.ndarray.argmax(y_test,axis=1)

In [122]:
y_test_new

array([7, 2, 1, ..., 4, 5, 6])

In [123]:
num=10000
r=0
w=0
for i in range(num):
        #print ('y_pred ',y_pred[i])
        #print ('labels ',labels[i])
        #without the use of all() returns error truth value of an array with more than one element is ambiguous
        if y_pred_new[i].all() == y_test_new[i].all():
        #if np.array_equal(y_pred[i],y_test[i]):
            r+=1
        else:
            w+=1
print ("tested ",  num, "digits")
print ("correct: ", r, "wrong: ", w, "error rate: ", float(w)*100/(r+w), "%")
print ("got correctly ", float(r)*100/(r+w), "%")

tested  10000 digits
correct:  9872 wrong:  128 error rate:  1.28 %
got correctly  98.72 %
